# Applied Quantum Computing Lab: Grover's Algorithm (SOLUTIONS)

This lab will guide you through implementing and understanding Grover's algorithm, one of the most important quantum algorithms that provides a quadratic speedup for search problems.

## Prerequisites
- Basic understanding of quantum gates (Hadamard, X, Z, CNOT)
- Familiarity with Python and Qiskit

## Learning Objectives
By completing this lab, you will:
1. Understand the fundamental concepts of Grover's search algorithm
2. Implement different components of the algorithm (oracle and diffusion operator)
3. Analyze how the algorithm's performance scales with problem size
4. Visualize probability amplification through iterations

## Introduction

Grover's algorithm, developed by Lov Grover in 1996, is a quantum algorithm designed to search through an unstructured database or solve an unstructured search problem. It provides a quadratic speedup over classical algorithms.

### Key Concepts:

1. **Problem Statement**: Imagine you have a list of N unsorted items and want to find a specific one that satisfies a particular condition.
   - Classical approach: Check items one by one (O(N) operations in the worst case)
   - Grover's approach: Find the answer in approximately O(√N) operations

2. **Quantum Advantage**: Grover's algorithm achieves this speedup by using quantum superposition and a technique called amplitude amplification to increase the probability of measuring the correct answer.

3. **Algorithm Steps**:
   - Initialize a quantum system in a superposition of all possible states
   - Apply an oracle function that marks the solution(s)
   - Apply the Grover diffusion operator that amplifies the amplitude of the marked state(s)
   - Repeat the oracle and diffusion operations approximately √N times
   - Measure the system to obtain the solution with high probability

Let's implement Grover's algorithm step by step.

## Exercise 1: Implementing a Simple Grover's Circuit

In this first exercise, we'll implement a simple version of Grover's algorithm to search for a specific state, |11⟩, in a 2-qubit system.

First, let's import the required libraries:

In [ ]:
# Import required libraries
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer, AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
from qiskit.quantum_info import Statevector

# Display matplotlib plots inline in the notebook
%matplotlib inline

### Task 1.1: Create a Basic Grover's Circuit

Create a quantum circuit with 2 qubits that implements Grover's algorithm to search for the state |11⟩.

Follow these steps:
1. Create a 2-qubit quantum circuit
2. Apply Hadamard gates to put qubits in superposition
3. Apply the oracle for marking |11⟩ (Hint: Use a CZ gate)
4. Apply the diffusion operator (this is the most complex part)
   - Apply H gates to both qubits
   - Apply X gates to both qubits
   - Apply a controlled-Z operation (you can use H-CX-H sequence on target)
   - Undo the X gates
   - Undo the H gates
5. Add measurement operations

Complete the code below:

In [ ]:
# Grover's algorithm for 2 qubits — search for '11'
qc = QuantumCircuit(2)

# Step 1: Put the qubits in superposition
qc.h([0, 1])

# Step 2: Oracle that marks |11⟩ by flipping its phase
qc.cz(0, 1)  # Controlled-Z acts as oracle for |11⟩

# Step 3: Grover diffusion operator
# 3a: Apply H gates to transform back to computational basis
qc.h([0, 1])

# 3b: Apply X gates to flip all qubits
qc.x([0, 1])

# 3c: Apply a controlled phase flip (using H-CX-H)
qc.h(1)
qc.cx(0, 1)
qc.h(1)

# 3d: Undo the X gates
qc.x([0, 1])

# 3e: Apply H gates to transform back to superposition basis
qc.h([0, 1])

# Step 4: Measurement
qc.measure_all()

# Visualize the circuit
display(qc.draw('mpl'))

# Run the circuit
backend = Aer.get_backend('qasm_simulator')
result = backend.run(qc, shots=1024).result()
counts = result.get_counts()

# Print and visualize the results
print("Counts:", counts)
display(plot_histogram(counts, title="Grover's Algorithm Result for |11⟩"))

### Task 1.2: Understanding the Oracle

The oracle is a crucial component of Grover's algorithm. It marks the target state by flipping its phase from positive to negative without changing the probabilities.

**Questions:**

1. Why does the CZ gate mark the state |11⟩? Explain how this works.
2. If we wanted to mark the state |01⟩ instead, how would you modify the oracle?
3. What happens if we run Grover's algorithm without an oracle?

Write your answers below:

**Answers:**

1. The CZ (controlled-Z) gate applies a phase flip (multiplication by -1) to the target qubit only when the control qubit is in state |1⟩. When both qubits are in state |1⟩ (giving the state |11⟩), the phase gets flipped. The CZ gate doesn't change the probability of measuring any state, but it does change the phase of the |11⟩ state, marking it for amplification by the diffusion operator.

2. To mark the state |01⟩, we would need to:
   - Apply an X gate to qubit 0 (to flip it from |0⟩ to |1⟩)
   - Apply the CZ gate
   - Apply another X gate to qubit 0 to restore it
   
   This sequence temporarily transforms |01⟩ to |11⟩, applies the phase flip to |11⟩, and then transforms it back to |01⟩, effectively marking only the |01⟩ state.

3. Without an oracle, the diffusion operator would amplify the average amplitude across all states, which would not lead to any state being preferentially measured. The result would be a uniform distribution across all possible states, similar to the initial superposition. The oracle is essential because it creates the asymmetry that the diffusion operator can then amplify.

## Exercise 2: Creating Modular Functions for Grover's Algorithm

In real applications, we want to make our implementation more modular and reusable. In this exercise, we'll create functions to generate the oracle and diffusion operator for any number of qubits.

Complete the following functions:

In [ ]:
# Define the problem parameters
nb_qubits = 2  # We need 2 qubits to represent 4 items (00, 01, 10, 11)
target = '11'   # We're looking for this specific value

print(f"Problem: We have 2^{nb_qubits} = {2**nb_qubits} elements in our search space")
print(f"We want to find the element corresponding to the binary string: {target}")

# Function to create an oracle that marks the target state
def create_oracle(target_state, num_qubits):
    """
    Creates a quantum circuit that implements an oracle marking the target state
    
    Parameters:
    -----------
    target_state: str
        Binary string representing the target state (e.g., '11')
    num_qubits: int
        Number of qubits in the circuit
        
    Returns:
    --------
    QuantumCircuit
        The oracle circuit
    """
    oracle = QuantumCircuit(num_qubits, name="Oracle")
    
    # For each qubit that should be 0 in the target state, apply X gates before and after
    # This converts the target state to |11...1⟩ temporarily
    
    # First, apply X gates to bits that should be 0
    for i in range(num_qubits):
        if target_state[i] == '0':
            oracle.x(num_qubits - 1 - i)  # Adjust for endianness if needed
    
    # Apply a multi-controlled Z gate
    # For 2 qubits, we can use CZ directly
    if num_qubits == 2:
        oracle.cz(0, 1)
    else:
        # For more qubits, we'd need a more complex implementation
        # Here's a simple approach using H gates and CNOT chains
        oracle.h(num_qubits - 1)
        for i in range(num_qubits - 1):
            oracle.cx(i, num_qubits - 1)
        oracle.h(num_qubits - 1)
    
    # Apply X gates again to restore the qubits
    for i in range(num_qubits):
        if target_state[i] == '0':
            oracle.x(num_qubits - 1 - i)
            
    return oracle

# Function to create the diffusion operator
def create_diffusion(num_qubits):
    """
    Creates a quantum circuit that implements the Grover diffusion operator
    
    Parameters:
    -----------
    num_qubits: int
        Number of qubits in the circuit
        
    Returns:
    --------
    QuantumCircuit
        The diffusion operator circuit
    """
    diffusion = QuantumCircuit(num_qubits, name="Diffusion")
    
    # 1. Apply H gates to all qubits
    for qubit in range(num_qubits):
        diffusion.h(qubit)
    
    # 2. Apply X gates to all qubits
    for qubit in range(num_qubits):
        diffusion.x(qubit)
    
    # 3. Apply a multi-controlled Z gate
    if num_qubits == 2:
        # For 2 qubits, we can use: H on target, CX, H on target
        diffusion.h(1)
        diffusion.cx(0, 1)
        diffusion.h(1)
    else:
        # For more qubits, apply H to last qubit and CX from all others
        diffusion.h(num_qubits - 1)
        for control_qubit in range(num_qubits - 1):
            diffusion.cx(control_qubit, num_qubits - 1)
        diffusion.h(num_qubits - 1)
    
    # 4. Undo the X gates
    for qubit in range(num_qubits):
        diffusion.x(qubit)
    
    # 5. Undo the H gates
    for qubit in range(num_qubits):
        diffusion.h(qubit)
    
    return diffusion

# Test your functions - try creating an oracle for state |10⟩
test_oracle = create_oracle('10', nb_qubits)
print("Oracle for state |10⟩:")
display(test_oracle.draw('mpl'))

# Test your diffusion operator
test_diffusion = create_diffusion(nb_qubits)
print("Diffusion operator:")
display(test_diffusion.draw('mpl'))

### Task 2.2: Create the Full Grover's Algorithm

Now, create a function that builds the complete Grover's circuit using your oracle and diffusion functions. This function should:

1. Take the target state, number of qubits, and number of iterations as parameters
2. Initialize qubits in superposition
3. Apply the oracle and diffusion operators for the specified number of iterations
4. Add measurement operations

In [ ]:
def create_grover_circuit(target_state, num_qubits, iterations):
    """
    Creates a Grover's algorithm circuit for finding the target state
    
    Parameters:
    -----------
    target_state: str
        Binary string representing the target state (e.g., '11')
    num_qubits: int
        Number of qubits in the circuit
    iterations: int
        Number of Grover iterations to apply
        
    Returns:
    --------
    QuantumCircuit
        The complete Grover's algorithm circuit
    """
    # 1. Create a circuit with num_qubits qubits and classical bits
    grover = QuantumCircuit(num_qubits, num_qubits)
    
    # 2. Apply H gates to put qubits in superposition
    grover.h(range(num_qubits))
    
    # 3. Create the oracle and diffusion operators
    oracle = create_oracle(target_state, num_qubits)
    diffusion = create_diffusion(num_qubits)
    
    # 4. Apply iterations of the Grover operator (oracle followed by diffusion)
    for _ in range(iterations):
        # Apply oracle
        grover = grover.compose(oracle)
        # Apply diffusion
        grover = grover.compose(diffusion)
    
    # 5. Add measurement operations
    grover.measure(range(num_qubits), range(num_qubits))
    
    return grover

# Calculate optimal number of iterations
N = 2**nb_qubits  # Total states
num_solutions = 1  # We're looking for just one state
optimal_iterations = int(np.round(np.pi/4 * np.sqrt(N/num_solutions)))
print(f"Optimal number of iterations: {optimal_iterations}")

# Create the Grover's algorithm circuit
grover_circuit = create_grover_circuit(target='11', num_qubits=nb_qubits, iterations=optimal_iterations)

# Visualize the circuit (without measurements for readability)
circuit_to_draw = create_grover_circuit(target='11', num_qubits=nb_qubits, iterations=optimal_iterations)
circuit_to_draw.remove_final_measurements()
print("Grover's algorithm circuit (without measurements):")
display(circuit_to_draw.draw('mpl'))

# Run the circuit
simulator = Aer.get_backend('qasm_simulator')
result = simulator.run(grover_circuit, shots=1024).result()
counts = result.get_counts()

# Display results and analyze performance
print("Measurement counts:", counts)
display(plot_histogram(counts, title="Grover's Algorithm Result"))

# Check if we found the right answer
max_result = max(counts, key=counts.get)
print(f"Most frequently measured state: |{max_result}⟩")
print(f"Target state: |{target}⟩")
print(f"Success: {'Yes' if max_result == target else 'No'}")
print(f"Probability of measuring the correct answer: {counts.get(target, 0)/1024:.4f} ({counts.get(target, 0)} out of 1024 shots)")

## Exercise 3: Visualizing Probability Amplification

One of the most fascinating aspects of Grover's algorithm is how it systematically increases the probability of measuring the target state with each iteration, up to an optimal point.

Complete the function below to simulate Grover's algorithm with different numbers of iterations and visualize how the probability of measuring the target state changes.

In [ ]:
def simulate_iterations(target_state, num_qubits, max_iterations):
    """
    Simulates Grover's algorithm with different numbers of iterations and
    returns the probability of measuring the target state for each iteration
    """
    probs = []
    
    # For each iteration count from 0 to max_iterations
    for i in range(max_iterations + 1):  # +1 to include initial state
        # Create a circuit to apply i iterations
        circuit = QuantumCircuit(num_qubits)
        circuit.h(range(num_qubits))
        
        oracle = create_oracle(target_state, num_qubits)
        diffusion = create_diffusion(num_qubits)
        
        for _ in range(i):  # Apply i iterations
            circuit = circuit.compose(oracle)
            circuit = circuit.compose(diffusion)
        
        # Get the statevector
        state = Statevector.from_instruction(circuit)
        
        # Calculate probability of the target state
        target_index = int(target_state, 2)
        prob = state.probabilities()[target_index]
        
        probs.append(prob)
        
    return probs

# Simulate up to 5 iterations
max_iterations = 5
probs = simulate_iterations(target='11', num_qubits=nb_qubits, max_iterations=max_iterations)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(range(max_iterations + 1), probs, marker='o', markersize=10, linewidth=2)
plt.grid(True)
plt.xlabel('Number of Iterations', fontsize=12)
plt.ylabel('Probability of Measuring Target State', fontsize=12)
plt.title('Probability Amplification in Grover\'s Algorithm', fontsize=14)

# Add horizontal line at 0.25 (initial probability)
plt.axhline(y=1/2**nb_qubits, color='r', linestyle='--', 
            label=f'Classical Probability (1/{2**nb_qubits})')

# Add an annotation for the optimal number of iterations
opt_iters = int(np.round(np.pi/4 * np.sqrt(2**nb_qubits)))
opt_index = min(opt_iters, max_iterations)
plt.annotate(f'Theoretical Optimal: {opt_iters} iterations', 
             xy=(opt_index, probs[opt_index]), 
             xytext=(opt_index-1, probs[opt_index]-0.2),
             arrowprops=dict(facecolor='black', shrink=0.05))

plt.xticks(range(max_iterations + 1))
plt.ylim(-0.05, 1.05)
plt.legend()
plt.show()

print(f"Optimal number of iterations for {2**nb_qubits} elements: {opt_iters}")
print("Notice how the probability increases and then decreases. This is why we need to run")
print("the optimal number of iterations - not too many and not too few.")

## Exercise 4: Scaling of Grover's Algorithm

A key benefit of Grover's algorithm is its quadratic speedup over classical algorithms. In this exercise, you'll explore how Grover's algorithm scales with problem size.

### Task 4.1: Calculate Iteration Requirements

Complete the following function to calculate the optimal number of iterations for different search problem sizes:

In [ ]:
def calculate_optimal_iterations(problem_sizes):
    """
    Calculate optimal number of Grover iterations for different problem sizes
    
    Parameters:
    -----------
    problem_sizes: list
        List of problem sizes (number of items)
        
    Returns:
    --------
    tuple
        (classical_operations, quantum_operations)
        Lists of operation counts for classical and quantum approaches
    """
    classical_ops = []
    quantum_ops = []
    
    for size in problem_sizes:
        # Classical approach needs O(N) operations
        classical_ops.append(size)
        
        # Quantum approach needs O(sqrt(N)) operations
        # The optimal number of iterations is approximately π/4 * √N
        optimal_iterations = int(np.round(np.pi/4 * np.sqrt(size)))
        quantum_ops.append(optimal_iterations)
    
    return classical_ops, quantum_ops

# Define a range of problem sizes to analyze
problem_sizes = [4, 16, 64, 256, 1024, 4096, 16384, 65536, 1000000]

# Calculate optimal iterations
classical_ops, quantum_ops = calculate_optimal_iterations(problem_sizes)

# Create a comparison table
print("Scaling Comparison: Classical vs. Quantum (Grover's Algorithm)")
print("-" * 70)
print(f"{'Problem Size':<15} {'Classical Operations':<20} {'Quantum Operations':<20}")
print("-" * 70)
for i in range(len(problem_sizes)):
    print(f"{problem_sizes[i]:<15} {classical_ops[i]:<20} {quantum_ops[i]:<20}")

# Plot the scaling comparison
plt.figure(figsize=(12, 6))
plt.loglog(problem_sizes, classical_ops, 'r-', linewidth=2, marker='o', label='Classical Search O(N)')
plt.loglog(problem_sizes, quantum_ops, 'b-', linewidth=2, marker='s', label='Quantum Search O(√N)')

plt.title('Scaling of Classical Search vs. Quantum Search (Grover\'s Algorithm)', fontsize=14)
plt.xlabel('Problem Size (Number of Items)', fontsize=12)
plt.ylabel('Number of Operations', fontsize=12)
plt.legend()
plt.grid(True, which="both", ls="-")
plt.show()

## Challenge Exercise: Implementing a Custom Oracle

In real-world applications, oracles can be much more complex than simply marking a known state. They can encode sophisticated conditions or functions.

Create an oracle that marks solutions to a simple logical condition. For example, design an oracle that marks states where:
- The first bit is 1, AND
- The second bit is 0

This would mark the state |10⟩ in a 2-qubit system.

Then, use your oracle with the Grover algorithm framework you've built to find this state.

In [ ]:
# Implement a custom oracle for a logical condition
def create_custom_oracle(num_qubits):
    """
    Creates an oracle that marks states where:
    - The first bit is 1 AND
    - The second bit is 0
    
    In a 2-qubit system, this would mark state |10⟩
    """
    oracle = QuantumCircuit(num_qubits, name="Custom Oracle")
    
    # Apply X gate to second qubit (to detect '0')
    oracle.x(1)
    
    # Apply CZ to mark state where both conditions are met
    oracle.h(1)
    oracle.cx(0, 1)
    oracle.h(1)
    
    # Undo the X gate
    oracle.x(1)
    
    return oracle

# Create a full Grover circuit using the custom oracle
def create_custom_grover_circuit(num_qubits, iterations):
    """
    Creates a Grover's algorithm circuit using the custom oracle
    """
    grover = QuantumCircuit(num_qubits, num_qubits)
    
    # Put qubits in superposition
    grover.h(range(num_qubits))
    
    # Create the custom oracle and standard diffusion operator
    oracle = create_custom_oracle(num_qubits)
    diffusion = create_diffusion(num_qubits)
    
    # Apply iterations of the Grover operator
    for _ in range(iterations):
        grover = grover.compose(oracle)
        grover = grover.compose(diffusion)
    
    # Add measurement
    grover.measure(range(num_qubits), range(num_qubits))
    
    return grover

# Calculate optimal iterations (same as before for 2 qubits)
opt_iterations = int(np.round(np.pi/4 * np.sqrt(2**nb_qubits)))

# Create and run the circuit
custom_grover = create_custom_grover_circuit(nb_qubits, opt_iterations)
simulator = Aer.get_backend('qasm_simulator')
result = simulator.run(custom_grover, shots=1024).result()
counts = result.get_counts()

# Display results
print("Custom Oracle Results:")
display(plot_histogram(counts, title="Grover's Algorithm with Custom Oracle"))

# Verify the result
expected_state = '10'
max_result = max(counts, key=counts.get)
print(f"Most frequently measured state: |{max_result}⟩")
print(f"Expected state: |{expected_state}⟩")
print(f"Success: {'Yes' if max_result == expected_state else 'No'}")

## Reflection and Further Questions

1. How does the probability of finding the target state change as you increase the number of iterations beyond the optimal value? Why does this happen?

2. If you had a quantum computer with 100 qubits, what size of search problem could you theoretically solve with Grover's algorithm? How many iterations would be required?

3. What are some real-world applications where Grover's algorithm might provide meaningful speedup over classical approaches? Consider domains like cryptography, optimization, or database searching.

4. What are the limitations of Grover's algorithm? When would other quantum or classical algorithms be more appropriate?

## Reflection Answers

1. The probability oscillates as you increase iterations beyond the optimal value. This is because Grover's algorithm rotates the quantum state in a two-dimensional subspace. The optimal number of iterations rotates the state close to the target state, but continuing to rotate will eventually move it away from the target. This is why finding the correct number of iterations is crucial.

2. With 100 qubits, you could theoretically search a space of 2^100 ≈ 10^30 items. The optimal number of iterations would be approximately π/4 * √(2^100) ≈ π/4 * 2^50 ≈ 10^15 iterations. While this is a massive speedup over the classical 2^100 operations, it's still an enormous number of operations that would take a very long time even on a perfect quantum computer.

3. Real-world applications include:
   - Breaking symmetric cryptography (e.g., brute-forcing encryption keys)
   - Database search when no indexing is available
   - Constraint satisfaction problems 
   - Optimization problems that can be formulated as search problems
   - Solving systems of linear equations
   - Finding collisions in hash functions

4. Limitations of Grover's algorithm:
   - Requires a quantum oracle that can recognize the solution
   - Provides only quadratic speedup, which may not be enough for many problems
   - Requires knowing the exact number of solutions for optimal performance
   - Highly structured problems often have classical algorithms that outperform Grover's
   - When the search space is small or well-structured, classical algorithms with good data structures might be more practical

## Conclusion

In this lab, you've implemented Grover's search algorithm, one of the most important quantum algorithms that demonstrates a clear quantum advantage. You've learned:

1. How to implement the oracle and diffusion operators
2. How to build a complete Grover's circuit
3. How probability amplification works through iterations
4. How the algorithm scales with problem size

While the quadratic speedup might not seem as dramatic as other quantum algorithms like Shor's (which provides exponential speedup), Grover's algorithm is more broadly applicable since many problems can be reframed as search problems.

In the next lab, you'll explore another important quantum algorithm, the Variational Quantum Eigensolver (VQE), which has important applications in quantum chemistry and optimization.